In [1]:
import requests
import json
from APIKEY import eia_api
import pandas
import pymongo
import time

In [2]:
#connection strings for separate category and series api's
url_series = "http://api.eia.gov/series/?api_key="
url_category = "http://api.eia.gov/category/?api_key="

In [3]:
#example of request for category api
query_category_url = url_category + eia_api + "&category_id=240692"
request = requests.get(query_category_url)
request

<Response [200]>

In [4]:
#example of request for series api's
query_series_url = url_series + eia_api + "&series_id=PET.EMD_EPD2DM10_PTE_NUS_DPG.M"
request = requests.get(query_series_url)
request

<Response [200]>

In [5]:
#print an example of a request
weekly_price_by_product = request.json()
print(json.dumps(weekly_price_by_product, indent=4, sort_keys=True))

{
    "request": {
        "command": "series",
        "series_id": "PET.EMD_EPD2DM10_PTE_NUS_DPG.M"
    },
    "series": [
        {
            "copyright": "None",
            "data": [
                [
                    "200812",
                    null
                ],
                [
                    "200811",
                    2.773
                ],
                [
                    "200810",
                    3.472
                ],
                [
                    "200809",
                    3.931
                ],
                [
                    "200808",
                    4.194
                ],
                [
                    "200807",
                    4.631
                ],
                [
                    "200806",
                    4.592
                ],
                [
                    "200805",
                    4.355
                ],
                [
                    "200804",
                   

In [6]:
connection_string='mongodb://localhost:27017'
#connect to database function
def db_collection(connection_string):
    #connection_string = 'mongodb+srv://<favasiliu%40gmail.com>:<Gabriela1980>@cluster0-qoy1h.mongodb.net/test'
    client = pymongo.MongoClient(connection_string)
    #define database for storage
    db = client.eia_db
    #drop all stored data
    db.eia_data.drop()
    db.eia_data
    #define collection to store data
    eia_data_collection = db.eia_data
    return eia_data_collection
    #store data into database
    #eia_data_collection.insert_one(data)

In [18]:
#function with recursion to get all subdictionaries from EIA, and stores the results in the database
def EIA_data_collection(dict_var, iter_no, series_search_string, db_collection, start_time, address):
    #sleep to prevent server overload
    if (time.time() - start_time) % 600 > 500: 
        sleep = time.sleep(150)
        print(sleep)
    #db_collection.insert_one(dict_var)
    name = dict_var['name']
    childcategories = dict_var['childcategories']
    
    for i in range(0, len(childcategories)):
#        if childcategories[i]["name"].find(search_string) != -1:
        query_category_url = url_category + eia_api + "&category_id=" + str(childcategories[i]['category_id'])
        childcategories[i] = requests.get(query_category_url).json()['category']
        #update field in db
        #db.customers.update({first_name:"Steven"},{$set:{gender:"male"}};
        iter_no += 1
        print("dictvar", dict_var['category_id'])
        print("iter_no", iter_no)
        
        db_collection.update_many(
                {address+'childcategories.name': str(childcategories[i]['name'])},
                {"$set": { address+"childcategories.$": childcategories[i]}}
            )
        print("db = ",db_collection.find_one({'category_id' : childcategories[i]['category_id']}))
        #db_collection.update_one({"category_id": dict_var['category_id']}, {"$set" : dict_var})
        print(address)
        if address == "":
            address = address+"childcategories."+str(i)+"."
        else:
            address = address+"childcategories."+str(i)+"."
        dict_var['childcategories'][i] = \
            EIA_data_collection(childcategories[i], iter_no, series_search_string, db_collection, start_time, address)
            #EIA_data_collection(childcategories[i]['category'], iter_no, series_search_string, db_collection, start_time) 
            
    #db_collection.update_one({"category_id": dict_var['category_id']}, {"$set" : dict_var})   
    childseries = dict_var['childseries']
    for j in range(0, len(childseries)):
        #time.sleep(0.5)
        #extract only series that contain "weekly" in the name
        if childseries[j]['name'].find(series_search_string) != -1:
            query_series_url = url_series + eia_api + "&series_id=" + str(childseries[j]['series_id'])
            childseries[j] = requests.get(query_series_url).json()["series"][0]
            #update field in db
            addres1_child_series = address+'childseries.series_id'
            addres2_child_series = address+"childseries.$"
            print(addres2_child_series)
            db_collection.update_many(
                {address+'childseries.series_id': childseries[j]['series_id']},
                {"$set": { address+"childseries.$": childseries[j]}}
            )
            print("iter_no", iter_no)
            
        else:
            #remove series that are not in the filtering criteria
            db_collection.update_many(
                {address+'childseries.series_id': childseries[j]['series_id']},
                {"$unset": { address+"childseries.$": childseries[j]}}
            )
    
            iter_no += 1
            print("iter_no", iter_no)

    return dict_var
               

In [19]:
# May exceed data rate limit. open jupyter with the following command:
#jupyter notebook --NotebookApp.iopub_data_rate_limit=1.0e10

#define the query string with starting category_id
# query_category_url = url_category + eia_api + "&category_id=714757"
query_category_url = url_category + eia_api + "&category_id=240690"
request = requests.get(query_category_url)
weekly_price_by_product = request.json()

#starting timer for sleep (to prevent server overload)
start_time = time.time()

#store data needed and display format
eia_data_collection = db_collection(connection_string)
eia_data_collection.insert_one(weekly_price_by_product['category'])
data = EIA_data_collection(weekly_price_by_product['category'], 1, "", eia_data_collection, start_time, "")
#print(json.dumps(data, indent=4, sort_keys=True))

dictvar 240690
iter_no 2
db =  None
address= 
dictvar 240691
iter_no 3
db =  None
address= childcategories.0.
childcategories.0.childcategories.0.childseries.$
iter_no 3
childcategories.0.childcategories.0.childseries.$
iter_no 3
childcategories.0.childcategories.0.childseries.$
iter_no 3
childcategories.0.childcategories.0.childseries.$
iter_no 3
childcategories.0.childcategories.0.childseries.$
iter_no 3
childcategories.0.childcategories.0.childseries.$
iter_no 3
childcategories.0.childcategories.0.childseries.$
iter_no 3
childcategories.0.childcategories.0.childseries.$
iter_no 3
childcategories.0.childcategories.0.childseries.$
iter_no 3
childcategories.0.childcategories.0.childseries.$
iter_no 3
childcategories.0.childcategories.0.childseries.$
iter_no 3
childcategories.0.childcategories.0.childseries.$
iter_no 3
childcategories.0.childcategories.0.childseries.$
iter_no 3
childcategories.0.childcategories.0.childseries.$
iter_no 3
childcategories.0.childcategories.0.childseries.$


KeyboardInterrupt: 